In [185]:
#importing & downloading required libraries
import nltk
import copy
from random import randrange

import os.path
from node_of_tree import Node

In [161]:
#change the version to avoid unicode error
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [162]:
#download the resources
nltk.download('large_grammars')

[nltk_data] Downloading package large_grammars to /root/nltk_data...
[nltk_data]   Package large_grammars is already up-to-date!


True

In [163]:
#loading the grammar

#The ATIS CFG is available in the NLTK data package, together with 98 test sentences.

grammar = nltk.data.load("grammars/large_grammars/atis.cfg")
grammar

<Grammar with 5517 productions>

In [244]:
#initialization of the following variables:
  ### 'productions_all'-----> dictionary 
  ### 'sentences_all' ------> lists
  ### 'states_all' ---------> lists

productions = {}
sentences = []
states = []



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



# 1. -----> From the CFG grammar find the following:
            ### productions
            ### sentences
            ### states

for i in range(len(grammar.productions())):
    keys = grammar.productions()[i].lhs()
    keys = str(keys)
    value1 = list(grammar.productions()[i].rhs())
    for i in range(len(value1)):
        if isinstance(value1[i], (str, unicode)):
            value1[i]="'"+value1[i]+"'"
        else:
            value1[i]=str(value1[i])
    if keys not in states:
        states.append(keys)    
        productions[keys]=[value1]
    else:
        if value1 not in productions[keys]:
            productions[keys].append(value1)



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#

#rules for binarization (CNF form)

#Final rules like A -> a
#Rules like A -> BC will go to CNF
#no empty RHS


# 2. -------> >2 to 2(Binarization)

def binarization():
    #variable for int_counting
    int_count =0

    #Make a copy of 'productions', as it's length will change after Binarization
    temp_prod_dict = copy.deepcopy(productions)

    for keys in temp_prod_dict:
        values = temp_prod_dict[keys]

        for i in range(len(values)):

            # checking for >2 case rule violation
            if len(values[i]) > 2:
            
                for j in range(0, len(values[i]) - 2):

                    # replacing
                    if j==0:
                        new_list = []
                        new_list.append(productions[keys][i][0])  #appending the new_list

                        letter = 'A' + str(int_count)
                        new_list.append(letter)    #appending the new_list

                        int_count = int_count + 1  #incrementing the count variable
                        productions[keys][i] = new_list   #reassigning 
                    
                    # adding the new productions
                    else:
                        new_list = []
                        new_list.append(values[i][j])   #appending the new_list

                        letter = 'A' + str(int_count)
                        new_list.append(letter)    #appending the new_list

                        int_count = int_count + 1  #incrementing the count variable
                        productions.setdefault(key_new, []).append(new_list)  #appending
                    
                    states.append(letter)   #appending
                    
                    # saving letter copy
                    # will be useing in the next rule
                    key_new = copy.deepcopy(letter)
                    
                # the last two letters will remain the same
                productions[key_new] = []
                productions[key_new].append(values[i][-2:])



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



# 3 -------> Removing the unary (=1) productions (A -> B)

def remove_unary():

    #pointer variable
    flag = 1

    #variable for counting
    int_count = 0
    while flag:

        #increment the counting variable
        int_count = int_count + 1   

        #to check for no variable
        flag = 0

        #Make a copy of 'productions', as it's length will change after elimination
        temp_prod_dict = copy.deepcopy(productions)

        for keys in temp_prod_dict:

            values = temp_prod_dict[keys]

            for i in range(len(values)):

                # Check for the rule violation
                if len(values[i])==1 and values[i][0] in states and values[i][0]!= keys:
                    new_list = copy.deepcopy(values[i][0])    #copy values to list
                    productions[keys].remove(values[i])
                    value_new = productions[new_list]       #reassigning values after removing duplicates

                    for j in range(len(value_new)):
                        productions[keys].append(value_new[j])
        
        for keys in productions:
            values = productions[keys]

            for k in range(len(values)):

                # Checking for the rule violation
                if len(values[k])==1 and values[k][0] in states and values[k][0]!= productions[values[k][0]]:
                    flag = 1
                    break

            if flag:
                break


#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



# Print the list 'productions' by passing different rules and writing it into a file each time
def print_rules(a_rule_num):
   
    with open('/content/'+ a_rule_num +'.txt', 'w') as f:   #opening a new file to write

        #counting variable
        int_count = 0

        for keys in productions:
            values = productions[keys]

            for i in range(len(values)):

                new_string = ""
                int_count = int_count + 1

                new_string += keys + '->' + str(values[i]) + '\n'
                f.write(new_string)    #writing to the new file 

        print(int_count)



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



# removing the duplicate productions

def remove():
    
    #Make a copy of 'productions', as it's length will change after elimination
    temp_prod_dict = copy.deepcopy(productions)

    for keys in temp_prod_dict:

        values = temp_prod_dict[keys]
        new_list = []

        for i in values:

            if i in new_list:
                productions[keys].remove(i)

            else:
                new_list.append(i)



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



#Parse a test sentence using CYK algorithm   

def CKYparser(test_sentence):

    test_sentence_length = len(test_sentence)

    for i in range(len(test_sentence)):
        test_sentence[i] = "'" + test_sentence[i] + "'"

    #create a matrix for storage
    matrix_chart = [[[] for i in range(test_sentence_length)] for j in range(test_sentence_length)]

    #create a trace pointer for back tracing the matrix
    back_trace_pointer = [[[] for i in range(test_sentence_length)] for j in range(test_sentence_length)]

    for j in range(1, test_sentence_length):

        for keys in productions:
            val = productions[keys]

            for i in val:
                if len(i) == 1  and  test_sentence[j-1] == i[0]:
                    matrix_chart[j-1][j].append(keys)
                    back_trace_pointer[j - 1][j].append(Node(keys, None, None, test_sentence[j - 1]))
        
        #traversing back using back_trace_pointer
        for i in reversed(range(0, j-1)):

            for k in range(i+1, j):

                for keys in productions:
                    values = productions[keys]

                    for v in range(len(values)):
                        if len(values[v]) == 2:
                            first_val = values[v][0]
                            second_val = values[v][1]

                            if first_val in matrix_chart[i][k] and second_val in matrix_chart[k][j]:
                                matrix_chart[i][j].append(keys)

                                for p in back_trace_pointer[i][k]:

                                    for q in back_trace_pointer[k][j]:

                                        if p.root == first_val and q.root == second_val:
                                            back_trace_pointer[i][j].append(Node(keys, p, q, None))

    #return output                                       
    return back_trace_pointer[0][test_sentence_length-1]



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#


#Returns the possible trees 

def tree(root, inter_node):

    #using the class and it's objects
    if root.status:
        return '(' + root.root + ' ' + root.terminal + ')'
    
    #incrementing the LHS length
    l = inter_node + 2 + len(root.left.root) 

    #incrementing the RHS length
    r = inter_node + 2 + len(root.right.root) 

    #make left
    left = tree(root.left, l)

    #make right
    right = tree(root.right, r)

    return '(' + root.root + ' ' + left + '\n' + ' '*inter_node + right + ')'



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



#generate a integer random number between 0-97 -----> to find the test sentence
random_number = randrange(98)

print("Random sentence no. selected is ", random_number + 1)
print('\n\n')



#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#



#main function to find parse trees by calling other functions within itself
def main():

    #about cfg dataset
    print('\nTotal no. of Initial productions rules from ATIS dataset')
    print_rules('File 1.0')

    #binarization of the productions values
    print('\nTotal no. of Rules after the Binarization and removal of duplicates')
    binarization()
    #remove()
    print_rules('File 2.0')
    
    print('\nTotal no. of Final CNF form rules after unary productions removal')
    remove_unary()
    #remove()
    print_rules('File 3.0')

    #------------------------------------------------------------------------------------
    #####Cocke–Kasami-Younger Parser/CKY parser###################################
    #print the trees using CKY parser function, grammar and test sentence

    #load the test sentences
    s = nltk.data.load("grammars/large_grammars/atis_sentences.txt")

    #extracting the test sentences
    s = nltk.parse.util.extract_test_sentences(s)
    
    for sentence in s:
        sentences.append(sentence[0])
  
    #storing the random integer  
    sent_int_count = random_number

    #finding the test_sentence at depending upon the random integer 
    test_sentence = sentences[random_number]

    #call CKYparser and store in a reverse pointer
    back_trace_pointer = CKYparser(test_sentence)

    start = str(grammar.start())

    #pointer 
    flag = 0

    #counting variable
    int_count = 0
    
    sent_int_count = sent_int_count + 1

    #using the class Node in file node.py to generate and print parse trees
    for node in back_trace_pointer:
        if node.root == start:
            int_count = int_count + 1
            print("\n\n\nParse trees for sentence no. " , sent_int_count, ":\n")
            print(tree(node, 3))
            print("\n**-----**---------**---------**-----------**---------**---------**----------**---------**----------**---------**\n")
            
            flag = 1

    #check flag if test sentence is not found
    if flag==0:
            print('\n\n\n"""""""""""""Sentence NOT FOUND""""""""""""""""""\n')

    #reset sentence to blank
    sentence = ""

    #appending the blank sentence
    for i in test_sentence:
        sentence = sentence + i

    #printing the no. of trees generater by the CKY parser    
    print("\nSentence no." , sent_int_count, 'has', str(int_count), 'no. of parse trees')
                    

#calling the main function for execution
main()

Random sentence no. selected is  55




Total no. of Initial productions rules from ATIS dataset
5517

Total no. of Rules after the Binarization and removal of duplicates
13500

Total no. of Final CNF form rules after unary productions removal
20344



Parse trees for sentence no.  55 :

(SIGMA (NP_NN (ADJ_WPS 'what')
          (A5097 (NP_NNS 'flights')
                 (NOUN_NN 'leave')))
   (A6148 (NOUN_NP 'boston')
          (PP_NP (PREP_IN 'to')
                 (NOUN_NP 'pittsburgh'))))

**-----**---------**---------**-----------**---------**---------**----------**---------**----------**---------**




Parse trees for sentence no.  55 :

(SIGMA (NP_NN (NP_NNS (ADJ_WPS 'what')
                  (NOUN_NNS 'flights'))
          (NOUN_NN 'leave'))
   (A6148 (NOUN_NP 'boston')
          (PP_NP (PREP_IN 'to')
                 (NOUN_NP 'pittsburgh'))))

**-----**---------**---------**-----------**---------**---------**----------**---------**----------**---------**


Sentence no. 55 has 2